In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import random

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
labels = train[['excerpt','target','standard_error']]

In [ ]:
# keep the same observation with probability p or normal sample with probability 1-p

def custom_bootstrap(p,val,se):
    chance = random.random() # random number b/w 0 and 1
    if chance <= p:
        #keep same observation
        return val
    else:
        #sample an observation from N(target,se)
        return np.random.normal(val, se)

    
def generate_aug_target(df, prob):
    target_aug = []
    for index,row in df.iterrows():
        mu = row['target']
        se = row['standard_error']
        target_aug.append(custom_bootstrap(prob,mu,se))
    return target_aug

In [ ]:
# checking avg difference with the sample 
prob = 0.7 #conservative threshold
for i in range(1,10):
    print(np.mean(generate_aug_target(labels,prob))-np.mean(labels['target']))

In [ ]:
labels['target_aug'] = generate_aug_target(labels,prob)
print('Target skew: ' + str(labels['target'].skew()))
print('Augmented target skew: ' + str(labels['target_aug'].skew()) + '\n')

fig, ax =plt.subplots(1,2)
fig.tight_layout()
sns.distplot(labels['target'], ax=ax[0])
sns.distplot(labels['target_aug'], ax=ax[1])
fig.show()

In [ ]:
labels[['target','target_aug']].describe()